# Rebuilding the base overlay
The full vivado project can be found in https://github.com/Pieter-Berteloot/PYNQ_Video_overlay.
Once we succesfully rebuild the base over to our video_stream overlay, we can now test out if everything still works.

**Place the .bit and .tcl file in the following directory:**

\\192.168.2.99\xilinx\pynq\overlays\base


- The **bit file** can be found after generating the bitstream in:
    
   \boards\Pynq-Z1\base\base\base.runs\impl_1
    
    
- The **TCL file** after running the write_bd_tcl top.tcl command
    
    c:/Users/[user]/AppData/Roaming/Xilinx/Vivado/top.tcl


All the code here can be found in the **hdmi_introduction example notebook**:

# New Style HDMI input and Pixel Formatting

This notebook introduces the new features of PYNQ 2.0 for interacting the video pipeline. The API has been completely
redesigned with high performance image processing applications in mind.

To start with download the base overlay and instantate the HDMI input and output.

In [ ]:
from pynq import Overlay
from pynq import MMIO
from pynq.lib.video import *

base = Overlay("/home/xilinx/pynq/overlays/base/top.bit")
base.download()

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

## Getting started

First we'll use the default pixel format which is 24 bit-per-pixel BGR formatted data for ease of use with OpenCV. 

In [ ]:
hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
hdmi_out.start()

The monitor should turn on and show a blank screen. To pass the image data through we can tie the output to the input. The tie will last until we send something else to be displayed.

In [ ]:
hdmi_in.tie(hdmi_out)

While this provides for a fast way of passing video data through the pipeline there is no way to access or modify the frames. For that we a loop calling `readframe` and `writeframe`.

In [ ]:
import time

numframes = 600
start = time.time()

for _ in range(numframes):
    f = hdmi_in.readframe()
    hdmi_out.writeframe(f)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

Next we can start adding some OpenCV processing into the mix. For all of these examples we are going to use a lapcian gradient filter. The first loop is going to perform the grayscale conversion in software.

In [ ]:
import cv2
import numpy as np

numframes = 10
grayscale = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width),
                       dtype=np.uint8)
result = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width),
                    dtype=np.uint8)

start = time.time()

for _ in range(numframes):
    inframe = hdmi_in.readframe()
    cv2.cvtColor(inframe,cv2.COLOR_BGR2GRAY,dst=grayscale)
    inframe.freebuffer()
    cv2.Laplacian(grayscale, cv2.CV_8U, dst=result)

    outframe = hdmi_out.newframe()
    cv2.cvtColor(result, cv2.COLOR_GRAY2BGR,dst=outframe)
    hdmi_out.writeframe(outframe)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

## Cleaning up

Finally you must always stop the interfaces when you are done with them. Otherwise bad things can happen when the bitstream is reprogrammed. You can also use the HDMI interfaces in a context manager to ensure that the cleanup is always performed.

In [ ]:
hdmi_out.close()
hdmi_in.close()

## Gray-scale
Using the new infrastructure we can delegate the color conversion to the hardware as well as only passing a single grayscale pixel to and from the processor.

First reconfigure the pipelines in grayscale mode and tie the two together to make sure everything is working correctly.

In [ ]:
base.download()

hdmi_in.configure(PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
hdmi_out.start()

hdmi_in.tie(hdmi_out)

No we can rewrite the loop without the software colour conversion.

In [ ]:
start = time.time()

numframes = 30

for _ in range(numframes):
    inframe = hdmi_in.readframe()
    outframe = hdmi_out.newframe()
    cv2.Laplacian(inframe, cv2.CV_8U, dst=outframe)
    inframe.freebuffer()
    hdmi_out.writeframe(outframe)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

In [ ]:
hdmi_out.close()
hdmi_in.close()

## Other modes

There are two other 24 bit modes that are useful for interacting with PIL. The first is regular RGB mode.

In [ ]:
base.download()

hdmi_in.configure(PIXEL_BGR)
hdmi_out.configure(hdmi_in.mode, PIXEL_BGR)

hdmi_in.start()
hdmi_out.start()

hdmi_in.tie(hdmi_out)

This is useful for easily creating and displaying frames with Pillow

In [ ]:
import PIL.Image

frame = hdmi_in.readframe()
image = PIL.Image.fromarray(frame)
image

An alternative mode is YCbCr which is useful for some image processing algorithms or exporting JPEG files. Because we are not changing the number of bits per pixel we can update the colorspace of the input dynamically.

In [ ]:
hdmi_in.colorspace = COLOR_IN_YCBCR

It's probably worth updating the output colorspace as well to avoid the psychedelic  effects

In [ ]:
hdmi_out.colorspace = COLOR_OUT_YCBCR

Now we can use PIL to read in the frame an perform the conversion back for us.

In [ ]:
import PIL.Image

frame = hdmi_in.readframe()
image = PIL.Image.fromarray(frame, "YCbCr")
frame.freebuffer()
image.convert("RGB")

In [ ]:
hdmi_out.close()
hdmi_in.close()

## Next Steps

This notebook has only provided an overview of base overlay pipeline. One of the reasons for the changes was to make it easier to add hardware accelerated functions by supporting a wider range of pixel formats without software conversion and separating out the HDMI front end from the video DMA. Explore the code in pynq/lib/video.py for more details.